# 4. Predict-Image-Similar-FCNN-Binary[split]
For landmark-recognition-2019 algorithm validation

## Run name

In [1]:
import time

project_name = 'Google-LandMark-Rec2019'
model_name = 'Google-LandMark-Rec2019_3-Image-Similar-FCNN-Binary_20190511-164020_8393'
step_name = '4-Predict[split]_%s' % model_name
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: 4-Predict[split]_Google-LandMark-Rec2019_3-Image-Similar-FCNN-Binary_20190511-164020_8393_20190513-132618


## Important params

In [2]:
import multiprocessing

cpu_amount = multiprocessing.cpu_count()
print('cpu_amount: ', cpu_amount)

small_debug_rows = 10
big_debug_rows = 20
libary_batch_amount = 20
print('small_debug_rows:\t', small_debug_rows)
print('big_debug_rows:\t', big_debug_rows)
print('libary_batch_amount:\t', libary_batch_amount)

cpu_amount:  4
small_debug_rows:	 10
big_debug_rows:	 20
libary_batch_amount:	 20


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import math
import shutil
import zipfile
import pickle
import h5py

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [4]:
import keras
from keras.utils import Sequence
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.preprocessing.image import *
from keras.applications.inception_v3 import preprocess_input

Using TensorFlow backend.


In [5]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.05
session = tf.Session(config=config)

KTF.set_session(session )

## Project folders

In [6]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

vgg16_feature_file = os.path.join(feature_folder, 'feature_wrapper_171023.h5')
train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')
print(vgg16_feature_file)
print(train_csv_file)
print(test_csv_file)
print(sample_submission_folder)

/data/landmark-recognition-2019/feature/feature_wrapper_171023.h5
/data/landmark-recognition-2019/input/train.csv
/data/landmark-recognition-2019/input/test.csv
/data/landmark-recognition-2019/input/sample_submission.csv


## Functions

In [7]:
def pickle_dump(data, file):
    with open(file, 'wb') as f:
        pickle.dump(data, f)

def pickle_load(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
    return data

a = list(range(10))
print(a)
demo_file = os.path.join(os.getcwd(), 'temp', 'pickle_demo.pkl')
print(demo_file)
pickle_dump(a, demo_file)
new_a = pickle_load(demo_file)
print(new_a)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
/data/landmark-recognition-2019/temp/pickle_demo.pkl
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [8]:
%%time

def get_whole_classes_label(class_indices_data, item_classes_data):
    indices2class = {}
    for i, key in enumerate(class_indices_data.keys()):
#         print(key, class_indices_data[key])
        indices2class[class_indices_data[key]] = key
#         if i >= 5:
#             break
#     for i, key in enumerate(class_indices_data.keys()):
#         print(key, class_indices_data[key], '-->', class_indices_data[key], indices2class[class_indices_data[key]])
#         if i >= 10:
#             break
    print(item_classes_data.shape)
    whole_classes_data = np.zeros(item_classes_data.shape)
    for i, class_index in enumerate(item_classes_data):
        whole_classes_data[i] = indices2class[class_index]
#         print(i, class_index, '-->', indices2class[class_index])
#         if i >= 10:
#             break
    return whole_classes_data
# get_whole_classes_label(class_indices_train, item_classes_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 µs


## Load feature

In [9]:
!ls ./feature -hl

total 8.6G
-rw-r--r-- 1 ubuntu ubuntu 830K May  3 12:19 feature_VGG16_data_val_171023.h5
-rw-r--r-- 1 root   root     20 May 11 04:24 feature_VGG19_data_test_171023_class_indices.pkl
-rw-r--r-- 1 root   root   222M May 11 04:24 feature_VGG19_data_test_171023.h5
-rw-r--r-- 1 ubuntu ubuntu 631K May  6 17:05 feature_VGG19_data_train_00_171023_class_indices.pkl
-rw-r--r-- 1 ubuntu ubuntu  99M May  6 17:05 feature_VGG19_data_train_00_171023.h5
-rw-r--r-- 1 ubuntu ubuntu 634K May  6 17:30 feature_VGG19_data_train_01_171023_class_indices.pkl
-rw-r--r-- 1 ubuntu ubuntu  99M May  6 17:30 feature_VGG19_data_train_01_171023.h5
-rw-r--r-- 1 ubuntu ubuntu 634K May  6 18:01 feature_VGG19_data_train_02_171023_class_indices.pkl
-rw-r--r-- 1 ubuntu ubuntu  99M May  6 18:01 feature_VGG19_data_train_02_171023.h5
-rw-r--r-- 1 ubuntu ubuntu 631K May  6 18:33 feature_VGG19_data_train_03_171023_class_indices.pkl
-rw-r--r-- 1 ubuntu ubuntu  99M May  6 18:33 feature_VGG19_data_train_03_171023.h5
-rw-r--r-- 1 u

In [10]:
def load_feature(data_set_name, model_name, date_str, feature_amount):
#     data_set_name = 'train'
#     model_name = 'VGG19'
#     date_str = '171023'
    x_data_arr = []
    classes_data_arr = []
    index_data_arr = []
    for i in range(feature_amount):
        folder_name = 'data_%s_%02d' % (data_set_name, i)
        class_indices_file = os.path.join(cwd, 'feature', 'feature_{0}_{1}_{2}_class_indices.pkl'.format(model_name, folder_name, date_str))
        h5py_file_name = os.path.join(cwd, 'feature', 'feature_{0}_{1}_{2}.h5'.format(model_name, folder_name, date_str))
        if not os.path.exists(class_indices_file):
            print('File not exists', class_indices_file)
            continue
        if not os.path.exists(h5py_file_name):
            print('File not exists', h5py_file_name)
            continue
        print(class_indices_file)
        print(h5py_file_name)
        class_indices_data = pickle_load(class_indices_file)
        print(len(class_indices_data))

        with h5py.File(h5py_file_name, 'r') as h:
            item_x_data = np.array(h['x_data_%s_%02d' % (data_set_name, i)])
            item_classes_data = np.array(h['classes_data_%s_%02d' % (data_set_name, i)])
            item_index_data = np.array(h['index_data_%s_%02d' % (data_set_name, i)])
        print(item_x_data.shape)
        x_data_arr.append(item_x_data)
        item_y_data = get_whole_classes_label(class_indices_data, item_classes_data)
        print(item_y_data.shape, item_y_data[:10])
        classes_data_arr.append(item_y_data)
        index_data_arr.append(item_index_data)
    x_data = np.concatenate(x_data_arr, axis=0)
    y_data = np.concatenate(classes_data_arr, axis=0)
    idx_data = np.concatenate(index_data_arr, axis=0)
    print('*' * 60)
    print(x_data.shape)
    print(y_data.shape)
    print(idx_data.shape)
    return x_data, y_data, idx_data


In [11]:
x_train, y_train, idx_train = load_feature('train', 'VGG19', '171023', libary_batch_amount)

/data/landmark-recognition-2019/feature/feature_VGG19_data_train_00_171023_class_indices.pkl
/data/landmark-recognition-2019/feature/feature_VGG19_data_train_00_171023.h5
34008
(50000, 512)
(50000,)
(50000,) [ 0.  3. 24. 25. 27. 27. 27. 27. 48. 52.]
/data/landmark-recognition-2019/feature/feature_VGG19_data_train_01_171023_class_indices.pkl
/data/landmark-recognition-2019/feature/feature_VGG19_data_train_01_171023.h5
34214
(50000, 512)
(50000,)
(50000,) [ 1. 12. 27. 27. 27. 27. 27. 27. 27. 37.]
/data/landmark-recognition-2019/feature/feature_VGG19_data_train_02_171023_class_indices.pkl
/data/landmark-recognition-2019/feature/feature_VGG19_data_train_02_171023.h5
34203
(50000, 512)
(50000,)
(50000,) [12. 17. 18. 27. 27. 27. 27. 27. 30. 35.]
/data/landmark-recognition-2019/feature/feature_VGG19_data_train_03_171023_class_indices.pkl
/data/landmark-recognition-2019/feature/feature_VGG19_data_train_03_171023.h5
34051
(50000, 512)
(50000,)
(50000,) [ 1.  5.  9. 10. 12. 27. 27. 27. 27. 27.]


In [12]:
x_val, y_val, idx_val = load_feature('val', 'VGG19', '171023', 4)

/data/landmark-recognition-2019/feature/feature_VGG19_data_val_00_171023_class_indices.pkl
/data/landmark-recognition-2019/feature/feature_VGG19_data_val_00_171023.h5
34160
(50000, 512)
(50000,)
(50000,) [ 1. 12. 20. 27. 27. 27. 27. 27. 27. 32.]
/data/landmark-recognition-2019/feature/feature_VGG19_data_val_01_171023_class_indices.pkl
/data/landmark-recognition-2019/feature/feature_VGG19_data_val_01_171023.h5
34196
(50000, 512)
(50000,)
(50000,) [ 9. 11. 24. 27. 27. 27. 27. 27. 27. 27.]
/data/landmark-recognition-2019/feature/feature_VGG19_data_val_02_171023_class_indices.pkl
/data/landmark-recognition-2019/feature/feature_VGG19_data_val_02_171023.h5
34264
(50000, 512)
(50000,)
(50000,) [ 1. 10. 11. 12. 27. 27. 27. 27. 27. 27.]
/data/landmark-recognition-2019/feature/feature_VGG19_data_val_03_171023_class_indices.pkl
/data/landmark-recognition-2019/feature/feature_VGG19_data_val_03_171023.h5
34155
(50000, 512)
(50000,)
(50000,) [ 1.  2. 10. 11. 13. 24. 27. 27. 27. 27.]
****************

## Load model

In [13]:
model_file = os.path.join(model_folder, '%s.h5' % model_name)
print(model_file)
model = load_model(model_file)

/data/landmark-recognition-2019/model/Google-LandMark-Rec2019_3-Image-Similar-FCNN-Binary_20190511-164020_8393.h5
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [14]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 512)          0                                            
__________________________________________________________________________________________________
library_input (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 1024)         0           main_input[0][0]                 
                                                                 library_input[0][0]              
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1024, 1)      0           concatenate_1[0][0]              
__________

## Predict

### single item

In [15]:
idx = 0
item_main_x = np.array([x_train[idx]]*x_train.shape[0])
item_x_train = {
    'main_input': item_main_x,
    'library_input': x_train
}
y_proba = model.predict(item_x_train, batch_size=1024)
print(int(y_train[idx]), np.argmax(y_proba), '-->', int(y_train[np.argmax(y_proba)]), [int(y_train[item[0]]) for item in np.argsort(y_proba, axis=0)[-10:]])

0 0 --> 0 [143204, 106644, 88680, 126637, 71016, 27725, 46205, 131056, 71615, 0]


### topn

In [16]:
topn = 10
amount = small_debug_rows
top1_count = 0
topn_count = 0
for idx in range(amount):
    t0 = time.time()
    item_main_x = np.array([x_train[idx]]*x_train.shape[0])
    item_x_train = {
        'main_input': item_main_x,
        'library_input': x_train
    }
    y_proba = model.predict(item_x_train, batch_size=1024)
    item_y_train = int(y_train[idx])
    top1_pred = int(y_train[np.argmax(y_proba)])
    topn_pred_arr = [int(y_train[item[0]]) for item in np.argsort(y_proba, axis=0)[-topn:]]
    if item_y_train == top1_pred:
        top1_count += 1
    if item_y_train in topn_pred_arr:
        topn_count += 1
    t1 = time.time()
    print('%4.2fs' % (t1 - t0), item_y_train, '-->', top1_pred, topn_pred_arr)

print('*' * 80)
print(top1_count, '%.2f' % (top1_count/amount))
print(topn_count, '%.2f' % (topn_count/amount))

7.92s 0 --> 0 [143204, 106644, 88680, 126637, 71016, 27725, 46205, 131056, 71615, 0]
7.97s 3 --> 74134 [151179, 192124, 107386, 176349, 40014, 132226, 166269, 3, 43351, 74134]
7.95s 24 --> 24 [11321, 32165, 193550, 138389, 129902, 201374, 163866, 112956, 109349, 24]
7.97s 25 --> 25 [86869, 202063, 244, 121194, 182724, 154318, 65855, 113168, 9604, 25]
7.97s 27 --> 27 [138760, 27, 88517, 27, 27, 27, 27, 27, 27, 27]
7.99s 27 --> 27 [27, 27, 27, 193759, 50990, 27, 68177, 27, 27, 27]
7.96s 27 --> 27 [122096, 27, 113636, 44570, 27, 27, 23830, 27, 27, 27]
7.96s 27 --> 27 [19095, 52773, 27, 77203, 48701, 27, 27, 27, 37837, 27]
7.96s 48 --> 48 [141654, 55177, 161939, 71555, 150738, 100982, 59663, 99546, 155715, 48]
7.97s 52 --> 52 [67091, 65218, 150691, 17212, 20740, 41648, 191292, 108327, 5561, 52]
********************************************************************************
9 0.90
10 1.00


### weighted topn

In [17]:
def get_weight_topn(y_proba, topn, y_data):
    y_proba_argsorted = [(y_data[item[0]], y_proba[item[0]][0]) for item in np.argsort(y_proba, axis=0)[-topn:]]
    class_score_dict = {}
    for item_class, item_proba in y_proba_argsorted:
        if item_class in class_score_dict:
            class_score_dict[item_class] += item_proba
        else:
            class_score_dict[item_class] = item_proba
    class_score_arr = list(class_score_dict.items())
    class_score_arr = sorted(class_score_arr, key=lambda x: x[1])
    topn_pred_arr = [int(item[0]) for item in class_score_arr]
    class_score_arr = [round(item[1], 3) for item in class_score_arr]
    return topn_pred_arr, class_score_arr

y_proba = np.array([[0.1], [0.2], [0.3], [0.2], [0.2], [0.2]])
topn = 6
y_data = np.array([0, 0, 1, 1, 2, 2])
print(y_proba, type(y_proba))
print(topn)
print(y_data, type(y_data))
get_weight_topn(y_proba, topn, y_data)

[[0.1]
 [0.2]
 [0.3]
 [0.2]
 [0.2]
 [0.2]] <class 'numpy.ndarray'>
6
[0 0 1 1 2 2] <class 'numpy.ndarray'>


([0, 2, 1], [0.3, 0.4, 0.5])

In [18]:
topn = 10
amount = small_debug_rows
top1_count = 0
topn_1_count = 0
topn_count = 0
for idx in range(amount):
    item_main_x = np.array([x_train[idx]]*x_train.shape[0])
    item_x_train = {
        'main_input': item_main_x,
        'library_input': x_train
    }
    y_proba = model.predict(item_x_train, batch_size=1024)
    item_y_train = int(y_train[idx])
    top1_pred = int(y_train[np.argmax(y_proba)])
#     topn_pred_arr = [y_train[item[0]] for item in np.argsort(y_proba, axis=0)[-topn:]]
    topn_pred_arr, class_score_arr = get_weight_topn(y_proba, topn, y_train)
    topn_1_pred = topn_pred_arr[-1]
    if item_y_train == top1_pred:
        top1_count += 1
    if item_y_train == topn_1_pred:
        topn_1_count += 1
    if item_y_train in topn_pred_arr:
        topn_count += 1
    print(item_y_train, '-->', top1_pred, '\t', topn_1_pred, topn_pred_arr, class_score_arr)

print('*' * 80)
print(top1_count, '%.2f' % (top1_count/amount))
print(topn_1_count, '%.2f' % (topn_1_count/amount))
print(topn_count, '%.2f' % (topn_count/amount))

0 --> 0 	 0 [143204, 106644, 88680, 126637, 71016, 27725, 46205, 131056, 71615, 0] [0.978, 0.979, 0.98, 0.981, 0.981, 0.981, 0.982, 0.984, 0.986, 0.99]
3 --> 74134 	 74134 [151179, 192124, 107386, 176349, 40014, 132226, 166269, 3, 43351, 74134] [0.974, 0.975, 0.975, 0.976, 0.976, 0.976, 0.977, 0.979, 0.98, 0.981]
24 --> 24 	 24 [11321, 32165, 193550, 138389, 129902, 201374, 163866, 112956, 109349, 24] [0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999, 0.999]
25 --> 25 	 25 [86869, 202063, 244, 121194, 182724, 154318, 65855, 113168, 9604, 25] [0.991, 0.992, 0.992, 0.992, 0.992, 0.993, 0.994, 0.994, 0.995, 0.997]
27 --> 27 	 27 [138760, 88517, 27] [0.997, 0.997, 7.981]
27 --> 27 	 27 [193759, 50990, 68177, 27] [0.996, 0.996, 0.998, 6.98]
27 --> 27 	 27 [122096, 113636, 44570, 23830, 27] [0.99, 0.99, 0.991, 0.992, 5.96]
27 --> 27 	 27 [19095, 52773, 77203, 48701, 37837, 27] [0.995, 0.995, 0.996, 0.996, 0.997, 4.984]
48 --> 48 	 48 [141654, 55177, 161939, 71555, 150738, 100982

### encupsolution

In [19]:
def get_single_pred(idx, main_x, libary_x, topn, batch_size=1024):
    item_main_x = np.array([main_x[idx]]*libary_x.shape[0])
    item_x = {
        'main_input': item_main_x,
        'library_input': libary_x
    }
    y_proba = model.predict(item_x, batch_size=batch_size)
    top1_pred = int(y_train[np.argmax(y_proba)])
    topn_pred_arr = [int(y_train[item[0]]) for item in np.argsort(y_proba, axis=0)[-topn:]]
    weighted_topn_pred_arr, class_score_arr = get_weight_topn(y_proba, topn, y_train)
    weighted_top1_pred = weighted_topn_pred_arr[-1]
    return top1_pred, topn_pred_arr, weighted_top1_pred, weighted_topn_pred_arr, class_score_arr

idx = 0
topn = 10
# train
print(int(y_train[idx]), get_single_pred(idx, x_train, x_train, topn, batch_size=1024))
# val
print(int(y_val[idx]), get_single_pred(idx, x_val, x_train, topn, batch_size=1024))

0 (0, [143204, 106644, 88680, 126637, 71016, 27725, 46205, 131056, 71615, 0], 0, [143204, 106644, 88680, 126637, 71016, 27725, 46205, 131056, 71615, 0], [0.978, 0.979, 0.98, 0.981, 0.981, 0.981, 0.982, 0.984, 0.986, 0.99])
1 (169233, [62074, 198944, 165771, 4822, 188696, 185275, 176528, 156620, 196838, 169233], 169233, [62074, 198944, 165771, 4822, 188696, 185275, 176528, 156620, 196838, 169233], [0.971, 0.972, 0.972, 0.973, 0.974, 0.975, 0.975, 0.976, 0.976, 0.98])


In [20]:
topn = 100
amount = big_debug_rows
top1_count = 0
topn_1_count = 0
topn_count = 0
for idx in range(amount):
    top1_pred, topn_pred_arr, weighted_top1_pred, weighted_topn_pred_arr, class_score_arr = get_single_pred(idx, x_train, x_train, topn, batch_size=1024)
    
    item_y_data = int(y_train[idx])
    if item_y_data == top1_pred:
        top1_count += 1
    if item_y_data == weighted_top1_pred:
        topn_1_count += 1
    if item_y_data in topn_pred_arr:
        topn_count += 1
    print(item_y_data, '-->', top1_pred, weighted_top1_pred, '\t', weighted_topn_pred_arr, class_score_arr)

print('*' * 80)
print(top1_count, '%.2f' % (top1_count/amount))
print(topn_1_count, '%.2f' % (topn_1_count/amount))
print(topn_count, '%.2f' % (topn_count/amount))

0 --> 0 27725 	 [160721, 162324, 45274, 45952, 104519, 115348, 128393, 33626, 114491, 3236, 90412, 25980, 165649, 55305, 92225, 47144, 14915, 186280, 160768, 124545, 135395, 43832, 185205, 89142, 15557, 81674, 136093, 158745, 132226, 43959, 28503, 184786, 57892, 118952, 112774, 173056, 40182, 49649, 179924, 190660, 199923, 173007, 102867, 111689, 152017, 4317, 78361, 22943, 33023, 55347, 137616, 106318, 114058, 7851, 64963, 150881, 107706, 152980, 50798, 129975, 47551, 38610, 8444, 26105, 35221, 6131, 99702, 43175, 57505, 73879, 1706, 7907, 44765, 62554, 194505, 146359, 76815, 44368, 14905, 88680, 126637, 71016, 46205, 131056, 71615, 0, 167988, 39592, 127989, 143204, 189620, 106644, 27725] [0.967, 0.967, 0.967, 0.967, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.969, 0.969, 0.969, 0.969, 0.969, 0.969, 0.969, 0.969, 0.969, 0.969, 0.969, 0.969, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.971, 0.971, 0.971, 0.972, 0.972,

48 --> 48 161939 	 [18346, 96986, 90736, 26882, 20946, 158745, 179793, 189201, 9775, 62742, 33716, 167169, 14567, 129135, 33966, 7492, 167121, 177870, 104616, 92419, 42071, 27493, 1860, 93073, 162027, 159192, 30030, 121873, 50049, 159094, 113227, 106109, 158804, 74025, 34512, 76537, 188159, 8064, 1078, 181774, 126637, 87774, 48333, 2948, 110718, 199506, 11048, 155802, 105759, 5138, 87420, 200380, 106716, 34866, 9744, 174827, 120552, 119699, 155042, 162300, 179550, 142211, 17312, 47923, 182506, 197667, 105214, 18318, 191296, 157486, 15968, 22237, 114187, 116228, 169572, 162005, 95821, 176137, 122827, 123211, 81039, 141654, 71555, 150738, 100982, 59663, 155715, 48, 42307, 121055, 73510, 55177, 99546, 161939] [0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.991, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.992, 0.9

98 --> 98 138982 	 [88546, 103588, 36798, 191009, 72615, 116321, 161668, 156657, 91412, 97548, 140248, 36290, 29060, 111699, 191292, 87005, 147369, 167988, 57015, 131040, 23582, 202225, 198232, 20168, 28163, 380, 141326, 150105, 154320, 19369, 82579, 52492, 188206, 119199, 93521, 188787, 200945, 119931, 131030, 139858, 103859, 2586, 141327, 50973, 23051, 176457, 37835, 113750, 189967, 11393, 40877, 3494, 51018, 73879, 173078, 193610, 198824, 120986, 79844, 186645, 129769, 196974, 147843, 124149, 43847, 187463, 117041, 75805, 124726, 88802, 100764, 56827, 32075, 3057, 167957, 96943, 169250, 98, 144201, 48580, 181151, 150591, 138982] [0.933, 0.933, 0.934, 0.934, 0.934, 0.934, 0.934, 0.934, 0.934, 0.934, 0.935, 0.936, 0.936, 0.936, 0.937, 0.937, 0.937, 0.937, 0.937, 0.937, 0.937, 0.938, 0.938, 0.938, 0.939, 0.939, 0.939, 0.939, 0.939, 0.94, 0.94, 0.941, 0.941, 0.941, 0.941, 0.942, 0.942, 0.943, 0.943, 0.943, 0.943, 0.943, 0.943, 0.943, 0.944, 0.944, 0.944, 0.944, 0.945, 0.945, 0.946, 0.94

In [21]:
topn = 100
amount = big_debug_rows
top1_count = 0
topn_1_count = 0
topn_count = 0
for idx in range(amount):
    top1_pred, topn_pred_arr, weighted_top1_pred, weighted_topn_pred_arr, class_score_arr = get_single_pred(idx, x_val, x_train, topn, batch_size=1024)
    
    item_y_data = int(y_val[idx])
    if item_y_data == top1_pred:
        top1_count += 1
    if item_y_data == weighted_top1_pred:
        topn_1_count += 1
    if item_y_data in topn_pred_arr:
        topn_count += 1
    print(item_y_data, '-->', top1_pred, weighted_top1_pred, '\t', weighted_topn_pred_arr, class_score_arr)

print('*' * 80)
print(top1_count, '%.2f' % (top1_count/amount))
print(topn_1_count, '%.2f' % (topn_1_count/amount))
print(topn_count, '%.2f' % (topn_count/amount))

1 --> 169233 62074 	 [68899, 191845, 172766, 9425, 80019, 109382, 52382, 119301, 107917, 189304, 102898, 62798, 119521, 137270, 42169, 73772, 71023, 188916, 166977, 142452, 36557, 130978, 158782, 23209, 153354, 185311, 164038, 155585, 157120, 196640, 198527, 166015, 64897, 4728, 162549, 198958, 98057, 180441, 68644, 157352, 5719, 145055, 70294, 2419, 194264, 76602, 118092, 57688, 148494, 126755, 78801, 100511, 21079, 167917, 178300, 66438, 27043, 134582, 131841, 178826, 139442, 186868, 123202, 82259, 157541, 41505, 5234, 1497, 148648, 107332, 201723, 177987, 51641, 199945, 186080, 198944, 4822, 188696, 185275, 176528, 156620, 196838, 169233, 144313, 184313, 47712, 4891, 92881, 88596, 165771, 62074] [0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961, 0.962, 0.962, 0.962, 0.962, 0.962, 0.962, 0.962, 0.962, 0.962, 0.962, 0.962, 0.963, 0.963, 0.963, 0.963, 0.963, 0.963, 0.963, 0.963, 0.963, 0.963, 0.964, 0.964, 0.964, 0.964, 0.964, 0.964, 0.96

27 --> 47035 27 	 [163321, 107219, 193759, 64183, 180346, 171344, 184025, 202429, 454, 156993, 57927, 17721, 149487, 153608, 58966, 154026, 80831, 176899, 191603, 70909, 45639, 143319, 52090, 196104, 124000, 61505, 114361, 122096, 104039, 174512, 32338, 145990, 87157, 178480, 33515, 130293, 118979, 4783, 18950, 31898, 127664, 156045, 189907, 10419, 190665, 115209, 92369, 42547, 152837, 89833, 48409, 2003, 166934, 84194, 150977, 64792, 60894, 87707, 101936, 151408, 196812, 185510, 190929, 47378, 132192, 197047, 86283, 196886, 49960, 183170, 175431, 145949, 202080, 199450, 135054, 142558, 123738, 143362, 76200, 174834, 47035, 27] [0.963, 0.963, 0.963, 0.963, 0.963, 0.963, 0.963, 0.963, 0.964, 0.964, 0.964, 0.964, 0.964, 0.964, 0.964, 0.964, 0.965, 0.965, 0.965, 0.965, 0.965, 0.965, 0.965, 0.966, 0.966, 0.966, 0.966, 0.967, 0.967, 0.967, 0.967, 0.967, 0.967, 0.967, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.969, 0.969, 0.969, 0.969, 0.97, 0.97, 0.97, 0.971, 0.971, 0.971, 0.971, 0.971, 0.

87 --> 87 175431 	 [47880, 11719, 64673, 200231, 121008, 194406, 30130, 171303, 70336, 27947, 200068, 155286, 120556, 13471, 201982, 98724, 133454, 75285, 11656, 88483, 62843, 46113, 80866, 49952, 28444, 194151, 33636, 150977, 60820, 99772, 83631, 20755, 198124, 104245, 192497, 62914, 14065, 131755, 91563, 107386, 114464, 179486, 192401, 198912, 16963, 69379, 172952, 109928, 128748, 5205, 15880, 52640, 188344, 64792, 151635, 165596, 149446, 175360, 201051, 165692, 20516, 30305, 124838, 173511, 161718, 201174, 185909, 19172, 41167, 31927, 75655, 34698, 28410, 134316, 87, 67619, 183170, 92349, 21613, 113636, 163896, 145949, 175402, 176899, 60730, 175431] [0.981, 0.981, 0.981, 0.981, 0.981, 0.981, 0.981, 0.981, 0.981, 0.981, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.982, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 0.984, 0.984, 0.

In [22]:
acc10000 = int(topn_count/amount*10000)
# acc10000 = 10000 # for test
# print(acc10000)
if acc10000 == 10000:
    run_name_acc = '%s_%05d' % (run_name, acc10000)
else:
    run_name_acc = '%s_%04d' % (run_name, acc10000)
print(run_name_acc)

4-Predict[split]_Google-LandMark-Rec2019_3-Image-Similar-FCNN-Binary_20190511-164020_8393_20190513-132618_5500


In [23]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name_acc)

Time elapsed: 433.5s
4-Predict[split]_Google-LandMark-Rec2019_3-Image-Similar-FCNN-Binary_20190511-164020_8393_20190513-132618_5500
